1.3 LSH


In [1]:
import pandas as pd
import random
import hashlib
from collections import defaultdict

# Carica il dataset
file_path = r'C:\Users\gallo\Downloads\archive\rating.csv'  # Cambia con il tuo percorso
data = pd.read_csv(file_path)

# Raggruppa i film valutati da ciascun utente
user_movies = data.groupby('userId')['movieId'].apply(set).to_dict()

# Funzione per generare firme MinHash
def minhash_signature(user_movies, num_hashes):
    """
    Genera una firma MinHash per un utente.
    - user_movies: Set di ID film valutati dall'utente.
    - num_hashes: Numero di funzioni hash da usare.
    """
    signature = []
    for i in range(num_hashes):
        random.seed(i)  # Per ottenere risultati ripetibili
        a, b = random.randint(1, 100003), random.randint(0, 100003)
        hash_func = lambda x: (a * x + b) % 100003  # Funzione hash casuale
        min_hash = min(hash_func(movie) for movie in user_movies)
        signature.append(min_hash)
    return signature



In [2]:
# Genera le firme MinHash per ogni utente
num_hashes = 70  # Numero di funzioni hash
user_signatures = {user: minhash_signature(movies, num_hashes) for user, movies in user_movies.items()}


In [3]:
import hashlib
from collections import defaultdict

def create_buckets(user_signatures, num_bands, rows_per_band):
    """
    Crea i bucket dividendo le firme MinHash in bande e hashando ogni banda.
    
    Parameters:
    - user_signatures: Dizionario {user_id: MinHash_signature}.
    - num_bands: Numero di bande in cui dividere le firme MinHash.
    - rows_per_band: Numero di righe per banda.
    
    Returns:
    - buckets: Dizionario {bucket_id: lista_di_utenti}.
    """
    buckets = defaultdict(list)  # Bucket per raggruppare gli utenti

    # Ciclo su ogni utente e la sua firma
    for user_id, signature in user_signatures.items():
        # Dividi la firma MinHash in bande
        for band in range(num_bands):
            # Estrai le righe della banda corrente
            start_idx = band * rows_per_band
            end_idx = start_idx + rows_per_band
            band_rows = tuple(signature[start_idx:end_idx])

            # Hasha la banda per creare un bucket ID unico
            band_hash = hashlib.sha1(str(band_rows).encode()).hexdigest()

            # Aggiungi l'utente al bucket corrispondente
            buckets[band_hash].append(user_id)
    
    return buckets


In [15]:
# Parametri LSH
num_bands = 10
rows_per_band = num_hashes // num_bands  # Ogni banda contiene (70/10=7) righe

# Creazione dei bucket
buckets = create_buckets(user_signatures, num_bands, rows_per_band)

# Controlla i bucket
for bucket_id, users in list(buckets.items())[30:45]:
    print(f"Bucket {bucket_id}: {users}")

Bucket 9666691962aa020f4102ecf15081c47a9607850b: [4]
Bucket 5665af988ec23a93a1cd222771b90669c6c748ae: [4]
Bucket 52806b80789dd257d0a4c061929fb7ff50ef144c: [4]
Bucket 88e105c6368bddc8cbc1b5bc6ffc024055fdb04b: [4]
Bucket 0279aaa3177091718f4e3beea41b4b07701cb31c: [4]
Bucket 491e5fa53217a860999e51967c4bd407f36a537a: [4]
Bucket f57b213967cc99ebf33dbaab8549e22b2c6b6af8: [4]
Bucket d2e847e28e7f05a0df0988896bd107630a5ff677: [4]
Bucket b3e0ba0812adc51574c0b8be6c821eecac04ecbd: [4]
Bucket 4c029243d72cb6f5ca159d48586f44dc59ef21a2: [4]
Bucket 131a4e354eec8e771c51645b88379658dcc7d299: [5]
Bucket 897cf3776a8e2baa44b18cdc5839fe03b41f770f: [5]
Bucket ce7288b4bd88e423989188c845de6f22a4ae7209: [5, 64263, 99407]
Bucket 011e2e1ba8890c8f75b060a1652c3bfccfcb5a99: [5]
Bucket f02d4e656d53167d0d0730d28b015c62ece19d1b: [5, 4436, 13416, 95909, 122340, 129977]


In [17]:
# Lista di utenti nel bucket
bucket_users = [5, 64263, 99407]

# Verifica dei film in comune
common_movies = set.intersection(*(user_movies[user] for user in bucket_users))

print(f"Film in comune tra gli utenti {bucket_users}: {common_movies}")


Film in comune tra gli utenti [5, 64263, 99407]: {1035, 1036, 11, 780, 141, 17, 788, 150, 1079, 440, 316, 318, 457, 587, 589, 590, 593, 595, 475, 480, 364, 500, 376, 377, 380}


In [22]:
user1, user2, user3 = 5, 64263, 99407
jaccard_12 = len(user_movies[user1].intersection(user_movies[user2])) / len(user_movies[user1].union(user_movies[user2]))
jaccard_13 = len(user_movies[user1].intersection(user_movies[user3])) / len(user_movies[user1].union(user_movies[user3]))
jaccard_23 = len(user_movies[user2].intersection(user_movies[user3])) / len(user_movies[user2].union(user_movies[user3]))

print(f"Jaccard Similarity (User {user1} - User {user2}): {jaccard_12}")
print(f"Jaccard Similarity (User {user1} - User {user3}): {jaccard_13}")
print(f"Jaccard Similarity (User {user2} - User {user3}): {jaccard_23}")


Jaccard Similarity (User 5 - User 64263): 0.3047619047619048
Jaccard Similarity (User 5 - User 99407): 0.3442622950819672
Jaccard Similarity (User 64263 - User 99407): 0.2706766917293233


In [12]:
# Calcoliamo quanti bucket hanno più di 1, più di 3 e più di 6 utenti
buckets_more_than_1 = sum(1 for users in buckets.values() if len(users) > 1)
buckets_more_than_3 = sum(1 for users in buckets.values() if len(users) > 3)
buckets_more_than_6 = sum(1 for users in buckets.values() if len(users) > 6)

# Risultati
{
    "Bucket con più di 1 utente": buckets_more_than_1,
    "Bucket con più di 3 utenti": buckets_more_than_3,
    "Bucket con più di 6 utenti": buckets_more_than_6
}
#len(buckets)

{'Bucket con più di 1 utente': 298110,
 'Bucket con più di 3 utenti': 140903,
 'Bucket con più di 6 utenti': 81810}

In [2]:
def find_top_2_similar_users_interactive(target_user, buckets, user_movies, num_bands, rows_per_band):
    while True:
        similar_users = set()
        
        # Cerca utenti simili nei bucket
        for bucket_id, users in buckets.items():
            if target_user in users:
                similar_users.update(users)
        
        # Rimuovi l'utente target dai possibili simili
        similar_users.discard(target_user)
        
        # Se trovi utenti simili, calcola la similarità Jaccard e restituisci i migliori
        if similar_users:
            jaccard_scores = {}
            for user in similar_users:
                intersection = len(user_movies[target_user].intersection(user_movies[user]))
                union = len(user_movies[target_user].union(user_movies[user]))
                jaccard_scores[user] = intersection / union
            
            # Ordina gli utenti per similarità Jaccard
            top_2_users = sorted(jaccard_scores.items(), key=lambda x: x[1], reverse=True)[:2]
            return top_2_users
        
        # Se non trovi utenti simili, chiedi nuovi parametri
        print(f"Nessun utente simile trovato per l'utente {target_user}.")
        print(f"Parametri attuali: num_bands={num_bands}, rows_per_band={rows_per_band}")
        
        # Richiedi nuovi parametri tramite input
        try:
            num_bands = int(input("Inserisci il nuovo numero di bande (num_bands): "))
            rows_per_band = int(input("Inserisci il nuovo numero di righe per banda (rows_per_band): "))
        except ValueError:
            print("Inserisci un numero valido.")
            continue
        
        # Opzionale: aggiungi qui un ricalcolo dei bucket se necessario con i nuovi parametri


In [3]:
# Parametri iniziali
num_bands = 10
rows_per_band = 70 // num_bands  # Calcolo iniziale delle righe per banda

# Esempio: Cerca utenti simili per l'utente 97
target_user = 97
top_2_similar_users = find_top_2_similar_users_interactive(target_user, buckets, user_movies, num_bands, rows_per_band)

if top_2_similar_users:
    print(f"I due utenti più simili a {target_user} sono:")
    for user, score in top_2_similar_users:
        print(f"Utente {user} con Similarità Jaccard: {score:.2f}")


NameError: name 'buckets' is not defined

In [8]:
def recommend_movies(target_user, similar_users, user_movies, num_recommendations=5):
    recommendations = {}
    for similar_user, _, _ in similar_users:
        for movie in user_movies[similar_user]:
            if movie not in user_movies[target_user]:
                recommendations[movie] = recommendations.get(movie, 0) + 1  # Incrementa il punteggio
    
    # Ordina i film in base alla frequenza e limita al numero desiderato
    return sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:num_recommendations]

# Genera raccomandazioni per User 1
recommendations = recommend_movies(190, similar_users, user_movies)
print("Raccomandazioni:", recommendations)
print(f"Similar Users for Target User {target_user}: {similar_users}")



Raccomandazioni: []


NameError: name 'target_user' is not defined